In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [3]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [4]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [5]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [6]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [7]:
# creating 2d and 4d training/testing dataset
batch_size = 64
train_iter = mx.io.NDArrayIter(X_train_array, Y_train_array, batch_size=batch_size, label_name='svm_label')
test_iter = mx.io.NDArrayIter(X_test_array, Y_test_array, batch_size=batch_size, label_name='svm_label')

train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

In [8]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)


# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')

In [9]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 100,     # number of data passes for training 
    learning_rate = 0.01 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, 64) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 132987.59 samples/sec	accuracy=0.840088
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.066
INFO:root:Epoch[0] Validation-accuracy=0.833594
INFO:root:Epoch[1] Batch [64]	Speed: 118498.20 samples/sec	accuracy=0.848633
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.069
INFO:root:Epoch[1] Validation-accuracy=0.830469
INFO:root:Epoch[2] Batch [64]	Speed: 114123.14 samples/sec	accuracy=0.849365
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.068
INFO:root:Epoch[2] Validation-accuracy=0.825000
INFO:root:Epoch[3] Batch [64]	Speed: 103186.74 samples/sec	accuracy=0.850342
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.072
INFO:root:Epoch[3] Validation-accuracy=0.819531
INFO:root:Epoch[4] Batch [64]	Speed: 109783.94 samples/sec	accuracy=0.850830
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=

INFO:root:Epoch[39] Time cost=0.049
INFO:root:Epoch[39] Validation-accuracy=0.800391
INFO:root:Epoch[40] Batch [64]	Speed: 159378.34 samples/sec	accuracy=0.849854
INFO:root:Epoch[40] Resetting Data Iterator
INFO:root:Epoch[40] Time cost=0.056
INFO:root:Epoch[40] Validation-accuracy=0.800391
INFO:root:Epoch[41] Batch [64]	Speed: 168847.24 samples/sec	accuracy=0.849854
INFO:root:Epoch[41] Resetting Data Iterator
INFO:root:Epoch[41] Time cost=0.051
INFO:root:Epoch[41] Validation-accuracy=0.800391
INFO:root:Epoch[42] Batch [64]	Speed: 157453.14 samples/sec	accuracy=0.849854
INFO:root:Epoch[42] Resetting Data Iterator
INFO:root:Epoch[42] Time cost=0.051
INFO:root:Epoch[42] Validation-accuracy=0.800391
INFO:root:Epoch[43] Batch [64]	Speed: 176406.43 samples/sec	accuracy=0.849854
INFO:root:Epoch[43] Resetting Data Iterator
INFO:root:Epoch[43] Time cost=0.046
INFO:root:Epoch[43] Validation-accuracy=0.800391
INFO:root:Epoch[44] Batch [64]	Speed: 164230.93 samples/sec	accuracy=0.849854
INFO:root

INFO:root:Epoch[79] Resetting Data Iterator
INFO:root:Epoch[79] Time cost=0.053
INFO:root:Epoch[79] Validation-accuracy=0.799219
INFO:root:Epoch[80] Batch [64]	Speed: 156584.90 samples/sec	accuracy=0.849854
INFO:root:Epoch[80] Resetting Data Iterator
INFO:root:Epoch[80] Time cost=0.057
INFO:root:Epoch[80] Validation-accuracy=0.799219
INFO:root:Epoch[81] Batch [64]	Speed: 136927.39 samples/sec	accuracy=0.849854
INFO:root:Epoch[81] Resetting Data Iterator
INFO:root:Epoch[81] Time cost=0.058
INFO:root:Epoch[81] Validation-accuracy=0.799219
INFO:root:Epoch[82] Batch [64]	Speed: 143602.37 samples/sec	accuracy=0.849854
INFO:root:Epoch[82] Resetting Data Iterator
INFO:root:Epoch[82] Time cost=0.052
INFO:root:Epoch[82] Validation-accuracy=0.799219
INFO:root:Epoch[83] Batch [64]	Speed: 154548.04 samples/sec	accuracy=0.849854
INFO:root:Epoch[83] Resetting Data Iterator
INFO:root:Epoch[83] Time cost=0.051
INFO:root:Epoch[83] Validation-accuracy=0.799219
INFO:root:Epoch[84] Batch [64]	Speed: 16704

In [10]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 100
mod.save(prefix, epoch)

# load model back
model_loaded = mx.model.FeedForward.load(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0100.params"


In [11]:
prob = model_loaded.predict(val_iters)
logging.info('Finish predict of one Layer SVM Network...')
# Check the accuracy from prediction
val_iters.reset()

y_batch = []
for dbatch in val_iters:
    label = dbatch.label[0].asnumpy()
    pad = val_iters.getpad()
    real_size = label.shape[0] - pad
    y_batch.append(label[0:real_size])
y = np.concatenate(y_batch)

# Confusion Matrix and report 
py = np.argmax(prob, axis=1)
acc1 = float(np.sum(py == y)) / len(y)
logging.info('Test accuracy = %f', acc1)
cfn_matrix=confusion_matrix(y,py)
print(classification_report (y,py))
print(cfn_matrix)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.798747


             precision    recall  f1-score   support

        0.0       0.27      0.17      0.21       404
        1.0       0.85      0.92      0.88      2150

avg / total       0.76      0.80      0.78      2554

[[  67  337]
 [ 177 1973]]


In [12]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=4)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.render('DNN + SVM')

'DNN + SVM.pdf'

In [13]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 78033.21 samples/sec	accuracy=0.826202
INFO:root:Epoch[0] Train-accuracy=0.833239
INFO:root:Epoch[0] Time cost=0.114
INFO:root:Epoch[0] Validation-accuracy=0.841797
INFO:root:Epoch[1] Batch [64]	Speed: 65499.27 samples/sec	accuracy=0.845913
INFO:root:Epoch[1] Train-accuracy=0.833239
INFO:root:Epoch[1] Time cost=0.114
INFO:root:Epoch[1] Validation-accuracy=0.841797
INFO:root:Epoch[2] Batch [64]	Speed: 67926.10 samples/sec	accuracy=0.845913
INFO:root:Epoch[2] Train-accuracy=0.833239
INFO:root:Epoch[2] Time cost=0.111
INFO:root:Epoch[2] Validation-accuracy=0.841797
INFO:root:Epoch[3] Batch [64]	Speed: 65015.66 samples/sec	accuracy=0.845913
INFO:root:Epoch[3] Train-accuracy=0.833239
INFO:root:Epoch[3] Time cost=0.117
INFO:root:Epoch[3] Validation-accuracy=0.841797
INFO:root:Epoch[4] Batch [64]	Speed: 70738.35 samples/sec	accuracy=0.845913
INFO:root:Epoch[4] Train-accuracy=0.833239
INFO:root:Epoch[4] Time cost=0.109
INFO:root:Epoch[4] Validation-accuracy

INFO:root:Epoch[40] Batch [64]	Speed: 74982.63 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Train-accuracy=1.000000
INFO:root:Epoch[40] Time cost=0.101
INFO:root:Epoch[40] Validation-accuracy=0.813672
INFO:root:Epoch[41] Batch [64]	Speed: 81803.84 samples/sec	accuracy=1.000000
INFO:root:Epoch[41] Train-accuracy=1.000000
INFO:root:Epoch[41] Time cost=0.097
INFO:root:Epoch[41] Validation-accuracy=0.814453
INFO:root:Epoch[42] Batch [64]	Speed: 85921.69 samples/sec	accuracy=1.000000
INFO:root:Epoch[42] Train-accuracy=1.000000
INFO:root:Epoch[42] Time cost=0.093
INFO:root:Epoch[42] Validation-accuracy=0.813672
INFO:root:Epoch[43] Batch [64]	Speed: 86390.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[43] Train-accuracy=1.000000
INFO:root:Epoch[43] Time cost=0.090
INFO:root:Epoch[43] Validation-accuracy=0.813672
INFO:root:Epoch[44] Batch [64]	Speed: 68541.00 samples/sec	accuracy=1.000000
INFO:root:Epoch[44] Train-accuracy=1.000000
INFO:root:Epoch[44] Time cost=0.132
INFO:root:Epoch[44]

INFO:root:Epoch[80] Batch [64]	Speed: 82333.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[80] Train-accuracy=1.000000
INFO:root:Epoch[80] Time cost=0.094
INFO:root:Epoch[80] Validation-accuracy=0.814844
INFO:root:Epoch[81] Batch [64]	Speed: 81869.72 samples/sec	accuracy=1.000000
INFO:root:Epoch[81] Train-accuracy=1.000000
INFO:root:Epoch[81] Time cost=0.095
INFO:root:Epoch[81] Validation-accuracy=0.814844
INFO:root:Epoch[82] Batch [64]	Speed: 83386.49 samples/sec	accuracy=1.000000
INFO:root:Epoch[82] Train-accuracy=1.000000
INFO:root:Epoch[82] Time cost=0.096
INFO:root:Epoch[82] Validation-accuracy=0.814844
INFO:root:Epoch[83] Batch [64]	Speed: 84363.09 samples/sec	accuracy=1.000000
INFO:root:Epoch[83] Train-accuracy=1.000000
INFO:root:Epoch[83] Time cost=0.096
INFO:root:Epoch[83] Validation-accuracy=0.815234
INFO:root:Epoch[84] Batch [64]	Speed: 81948.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[84] Train-accuracy=1.000000
INFO:root:Epoch[84] Time cost=0.094
INFO:root:Epoch[84]

Accuracy: 81.6015625 %


In [14]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 100
mod.save_checkpoint(prefix, epoch)

# load model back
model_loaded = mx.model.FeedForward.load(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0100.params"


In [15]:
prob = model_loaded.predict(val_iters)
logging.info('Finish predict of DNN+SVM')
# Check the accuracy from prediction
val_iters.reset()

y_batch = []
for dbatch in val_iters:
    label = dbatch.label[0].asnumpy()
    pad = val_iters.getpad()
    real_size = label.shape[0] - pad
    y_batch.append(label[0:real_size])
y = np.concatenate(y_batch)

# get prediction label 
py = np.argmax(prob, axis=1)
acc1 = float(np.sum(py == y)) / len(y)
logging.info('Test accuracy = %f', acc1)
cfn_matrix=confusion_matrix(y,py)
print(classification_report (y,py))
print(cfn_matrix)

INFO:root:Finish predict of DNN+SVM
INFO:root:Test accuracy = 0.815975


             precision    recall  f1-score   support

        0.0       0.34      0.17      0.22       404
        1.0       0.86      0.94      0.90      2150

avg / total       0.77      0.82      0.79      2554

[[  67  337]
 [ 133 2017]]


In [16]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=32)

mlp = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=mlp, shape=shape)
network_chart=mx.viz.plot_network(symbol=mlp, shape=shape)
network_chart.render('DDNN + SVM')

'DDNN + SVM.pdf'

In [17]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = mlp,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, 64),  
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(test_iter, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 40604.08 samples/sec	accuracy=0.834375
INFO:root:Epoch[0] Train-accuracy=0.833239
INFO:root:Epoch[0] Time cost=0.249
INFO:root:Epoch[0] Validation-accuracy=0.841797
INFO:root:Epoch[1] Batch [64]	Speed: 37239.71 samples/sec	accuracy=0.845913
INFO:root:Epoch[1] Train-accuracy=0.833239
INFO:root:Epoch[1] Time cost=0.206
INFO:root:Epoch[1] Validation-accuracy=0.841797
INFO:root:Epoch[2] Batch [64]	Speed: 37487.39 samples/sec	accuracy=0.845913
INFO:root:Epoch[2] Train-accuracy=0.833239
INFO:root:Epoch[2] Time cost=0.205
INFO:root:Epoch[2] Validation-accuracy=0.841797
INFO:root:Epoch[3] Batch [64]	Speed: 37682.09 samples/sec	accuracy=0.845913
INFO:root:Epoch[3] Train-accuracy=0.833239
INFO:root:Epoch[3] Time cost=0.204
INFO:root:Epoch[3] Validation-accuracy=0.841797
INFO:root:Epoch[4] Batch [64]	Speed: 37019.12 samples/sec	accuracy=0.845913
INFO:root:Epoch[4] Train-accuracy=0.833239
INFO:root:Epoch[4] Time cost=0.209
INFO:root:Epoch[4] Validation-accuracy

INFO:root:Epoch[40] Batch [64]	Speed: 34716.06 samples/sec	accuracy=0.845913
INFO:root:Epoch[40] Train-accuracy=0.833239
INFO:root:Epoch[40] Time cost=0.232
INFO:root:Epoch[40] Validation-accuracy=0.841797
INFO:root:Epoch[41] Batch [64]	Speed: 29701.12 samples/sec	accuracy=0.845913
INFO:root:Epoch[41] Train-accuracy=0.833239
INFO:root:Epoch[41] Time cost=0.226
INFO:root:Epoch[41] Validation-accuracy=0.841797
INFO:root:Epoch[42] Batch [64]	Speed: 40372.59 samples/sec	accuracy=0.845913
INFO:root:Epoch[42] Train-accuracy=0.833239
INFO:root:Epoch[42] Time cost=0.202
INFO:root:Epoch[42] Validation-accuracy=0.841797
INFO:root:Epoch[43] Batch [64]	Speed: 26484.84 samples/sec	accuracy=0.845913
INFO:root:Epoch[43] Train-accuracy=0.833239
INFO:root:Epoch[43] Time cost=0.267
INFO:root:Epoch[43] Validation-accuracy=0.841797
INFO:root:Epoch[44] Batch [64]	Speed: 24771.49 samples/sec	accuracy=0.845913
INFO:root:Epoch[44] Train-accuracy=0.833239
INFO:root:Epoch[44] Time cost=0.285
INFO:root:Epoch[44]

INFO:root:Epoch[80] Batch [64]	Speed: 28167.88 samples/sec	accuracy=0.845913
INFO:root:Epoch[80] Train-accuracy=0.833239
INFO:root:Epoch[80] Time cost=0.238
INFO:root:Epoch[80] Validation-accuracy=0.841797
INFO:root:Epoch[81] Batch [64]	Speed: 40111.58 samples/sec	accuracy=0.845913
INFO:root:Epoch[81] Train-accuracy=0.833239
INFO:root:Epoch[81] Time cost=0.219
INFO:root:Epoch[81] Validation-accuracy=0.841797
INFO:root:Epoch[82] Batch [64]	Speed: 24958.15 samples/sec	accuracy=0.845913
INFO:root:Epoch[82] Train-accuracy=0.833239
INFO:root:Epoch[82] Time cost=0.309
INFO:root:Epoch[82] Validation-accuracy=0.841797
INFO:root:Epoch[83] Batch [64]	Speed: 25232.23 samples/sec	accuracy=0.845913
INFO:root:Epoch[83] Train-accuracy=0.833239
INFO:root:Epoch[83] Time cost=0.296
INFO:root:Epoch[83] Validation-accuracy=0.841797
INFO:root:Epoch[84] Batch [64]	Speed: 35151.56 samples/sec	accuracy=0.845913
INFO:root:Epoch[84] Train-accuracy=0.833239
INFO:root:Epoch[84] Time cost=0.208
INFO:root:Epoch[84]

Accuracy: 84.1796875 %


In [18]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 100
mod.save_checkpoint(prefix, epoch)

# load model back
model_loaded = mx.model.FeedForward.load(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0100.params"


In [19]:
prob = model_loaded.predict(val_iters)
logging.info('Finish predict of DDNN + SVM...')
# Check the accuracy from prediction
val_iters.reset()

y_batch = []
for dbatch in val_iters:
    label = dbatch.label[0].asnumpy()
    pad = val_iters.getpad()
    real_size = label.shape[0] - pad
    y_batch.append(label[0:real_size])
y = np.concatenate(y_batch)

# get prediction label 
py = np.argmax(prob, axis=1)
acc1 = float(np.sum(py == y)) / len(y)
logging.info('Test accuracy = %f', acc1)
cfn_matrix=confusion_matrix(y,py)
print(classification_report (y,py))
print(cfn_matrix)

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.841817


             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       404
        1.0       0.84      1.00      0.91      2150

avg / total       0.71      0.84      0.77      2554

[[   0  404]
 [   0 2150]]
